In [1]:
import tensorflow as tf
import glob 
import os
import pandas as pd
import numpy as np

C:\anaconda\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)


In [7]:
dirList=os.listdir('c:\\FEST_train\\test')
for product_id in dirList:
    os.rename("C:\\FEST_train\\test\\"+product_id,"C:\\FEST_train\\train\\"+product_id )

In [8]:
a='10047545284'
lst=[]
newlst=[]
count=0
ncount=0
dirList=os.listdir('c:\\FEST_train\\train')
for product_id in dirList:
    
    if a==product_id[:11]:
        count+=1
        if count%5==0:
            os.renames("C:\\FEST_train\\train\\"+product_id,"C:\\FEST_train\\test\\"+product_id )
            ncount+=1
    if a!=product_id[:11]:
        lst.append(count)
        newlst.append(ncount)
        with open("c:\\GH\\output3.txt", "a") as f:
            f.write("%s\n " %a)
        a=product_id[:11]
        count=0
        ncount=0

In [13]:
def label_getter(path):
    raw_filenames=glob.glob(path)
    label=[]
    for i in raw_filenames:
        a=i.split('\\')[-1]
        b=a.split('_')[0]
        label.append(b)
    return label

In [9]:
sess=tf.InteractiveSession()

In [32]:
train_filenames = glob.glob('c:\\FEST_train\\train\\*.*')
train_labels=label_getter('c:\\FEST_train\\train\\*.*')
test_filenames = glob.glob('c:\\FEST_train\\test\\*.*')
test_labels=label_getter('c:\\FEST_train\\test\\*.*')
#print(train_labels)

dataset = tf.data.Dataset.from_tensor_slices((train_filenames, train_labels))
# or dataset = tf.data.TFRecordDataset(filenames)
#dataset = tf.data.Dataset.zip((filename,label))
dataset = dataset.shuffle(buffer_size=10000)


def parse_function(filename, label):
    image_string = tf.read_file(filename)
    label=tf.cast(label,tf.int32)
    one_hot = tf.one_hot(label, 153) #Num_classes=153
    label=tf.reshape(one_hot,[-1,153])
    # Don't use tf.image.decode_image, or the output shape will be undefined
    image = tf.image.decode_jpeg(image_string, channels=3)

    # This will convert to float values in [0, 1] or image = tf.cast(image, tf.float32)
    image = tf.image.convert_image_dtype(image, tf.float32)

    #image = tf.image.resize_images(image, [64, 64])
    return image, label


dataset = dataset.map(parse_function, num_parallel_calls=4)
dataset = dataset.batch(32)
dataset = dataset.repeat(15)
dataset= dataset.prefetch(1)
iterator = dataset.make_one_shot_iterator()

x,y = iterator.get_next()

In [51]:
def conv_net(x, n_classes, keep_prob, reuse, is_training):
    # Define a scope for reusing the variables
    with tf.variable_scope('ConvNet', reuse=reuse):
        # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
        # Reshape to match picture format [Height x Width x Channel]
        # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
        x = tf.reshape(x, shape=[-1, 300, 300, 3])

        
        W1 = tf.Variable(tf.random_normal([3, 3, 3, 32], stddev=0.01))

        L1 = tf.nn.conv2d(x, W1, strides=[1, 1, 1, 1], padding='SAME')
        L1 = tf.nn.relu(L1)
        L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1], padding='SAME')
        L1 = tf.layers.dropout(L1, rate=keep_prob, training=is_training)
        #or high level API : Convolution Layer with 32 filters and a kernel size of 5
        #conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        # conv1 = tf.layers.max_pooling2d(conv1, 2, 2)
        
        # L2 ImgIn shape=(?, 150, 150, 32)
        W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
        #    Conv      ->(?, 150, 150, 64)
        #    Pool      ->(?, 75, 75, 64)
        L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
        L2 = tf.nn.relu(L2)
        L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1], padding='SAME')
        L2 = tf.layers.dropout(L2, rate=keep_prob, training=is_training)
        '''
        Tensor("Conv2D_1:0", shape=(?, 14, 14, 64), dtype=float32)
        Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)
        Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)
        Tensor("dropout_1/mul:0", shape=(?, 7, 7, 64), dtype=float32)
        '''

        # L3 ImgIn shape=(?, 75, 75, 64)
        W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01))
        #    Conv      ->(?, 75, 75, 128)
        #    Pool      ->(?, 25, 25, 128)
        #    Reshape   ->(?, 25* 25* 128) # Flatten them for FC
        L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
        L3 = tf.nn.relu(L3)
        L3 = tf.nn.max_pool(L3, ksize=[1, 3, 3, 1], strides=[
                            1, 3, 3, 1], padding='SAME')
        L3 = tf.layers.dropout(L3, rate=keep_prob, training=is_training)
        L3_flat = tf.reshape(L3, [-1, 128 * 25 * 25])
        L3_flat=tf.cast(L3_flat,tf.float32)
        '''
        Tensor("Conv2D_2:0", shape=(?, 7, 7, 128), dtype=float32)
        Tensor("Relu_2:0", shape=(?, 7, 7, 128), dtype=float32)
        Tensor("MaxPool_2:0", shape=(?, 4, 4, 128), dtype=float32)
        Tensor("dropout_2/mul:0", shape=(?, 4, 4, 128), dtype=float32)
        Tensor("Reshape_1:0", shape=(?, 2048), dtype=float32)
        '''
        
        # L4 FC 4x4x128 inputs -> 625 outputs
        W4 = tf.get_variable("W4", shape=[128 * 25 * 25, 625],initializer=tf.contrib.layers.xavier_initializer())
        b4 = tf.Variable(tf.random_normal([625]))
        L4 = tf.nn.relu(tf.matmul(L3_flat, W4) + b4)
        L4 = tf.layers.dropout(L4, rate=keep_prob, training=is_training)
        '''
        Tensor("Relu_3:0", shape=(?, 625), dtype=float32)
        Tensor("dropout_3/mul:0", shape=(?, 625), dtype=float32)
        '''

        # L5 Final FC 625 inputs -> 10 outputs
        W5 = tf.get_variable("W5", shape=[625, 153],initializer=tf.contrib.layers.xavier_initializer())
        b5 = tf.Variable(tf.random_normal([153]))
        logits = tf.matmul(L4, W5) + b5
        return logits
        '''
        # Convolution Layer with 32 filters and a kernel size of 5
        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)

        # Flatten the data to a 1-D vector for the fully connected layer
        fc1 = tf.contrib.layers.flatten(conv2)

        # Fully connected layer (in contrib folder for now)
        fc1 = tf.layers.dense(fc1, 1024)
        # Apply Dropout (if is_training is False, dropout is not applied)
        fc1 = tf.layers.dropout(fc1, rate=keep_prob, training=is_training)

        # Output layer, class prediction
        out = tf.layers.dense(fc1, n_classes)
        # Because 'softmax_cross_entropy_with_logits' already apply softmax,
        # we only apply softmax to testing network
        out = tf.nn.softmax(out) if not is_training else out

    return out
    '''

In [53]:
logits_train = conv_net(x, 153, 0.8, reuse=tf.AUTO_REUSE, is_training=True)
# Create another graph for testing that reuse the same weights, but has
# different behavior for 'dropout' (not applied).
logits_test = conv_net(x, 153, 1, reuse=tf.AUTO_REUSE, is_training=False)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits_train, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits_test, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))



sess.run(tf.global_variables_initializer())
num_steps=2000
for step in range(1, num_steps + 1):

    # Run optimization
    sess.run(train_op)

    if step % display_step == 0 or step == 1:
        # Calculate batch loss and accuracy
        # (note that this consume a new batch of data)
        loss, acc = sess.run([loss_op, accuracy])
        print("Step " + str(step) + ", Minibatch Loss= " + \
              "{:.4f}".format(loss) + ", Training Accuracy= " + \
              "{:.3f}".format(acc))

print("Optimization Finished!")




Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



UnimplementedError: Cast string to int32 is not supported
	 [[node Cast (defined at <ipython-input-53-4da7393cc207>:13)  = Cast[DstT=DT_INT32, SrcT=DT_STRING, Truncate=false](arg1)]]
	 [[node OneShotIterator_1 (defined at <ipython-input-32-9a18d2e6d835>:32)  = OneShotIterator[container="", dataset_factory=_make_dataset_zW7uAmJq07g[], output_shapes=[[?,?,?,3], [?,1,153]], output_types=[DT_FLOAT, DT_FLOAT], shared_name="", _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Cast', defined at:
  File "C:\anaconda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\anaconda\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\anaconda\lib\site-packages\ipykernel\kernelapp.py", line 477,

In [2]:
# Get one and predict
r = random.randint(0, mnist.test.num_examples - 1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", sess.run(
    tf.argmax(logits, 1), feed_dict={X: mnist.test.images[r:r + 1], keep_prob: 1}))

# plt.imshow(mnist.test.images[r:r + 1].
#           reshape(28, 28), cmap='Greys', interpolation='nearest')
# plt.show()
loss = model_function(next_example, next_label)

training_op = tf.train.AdagradOptimizer(...).minimize(loss)

with tf.train.MonitoredTrainingSession(...) as sess:
    while not sess.should_stop():
        sess.run(training_op)

'123'

In [ ]:
learning_rate = 0.001
num_steps = 2000
batch_size = 128
display_step = 100

# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

sess = tf.Session()

# Create a dataset tensor from the images and the labels
dataset = tf.data.Dataset.from_tensor_slices(
    (mnist.train.images, mnist.train.labels))
# Automatically refill the data queue when empty
dataset = dataset.repeat()
# Create batches of data
dataset = dataset.batch(batch_size)
# Prefetch data for faster consumption
dataset = dataset.prefetch(batch_size)

# Create an iterator over the dataset
iterator = dataset.make_initializable_iterator()
# Initialize the iterator
sess.run(iterator.initializer)

# Neural Net Input (images, labels)
X, Y = iterator.get_next()



# Because Dropout have different behavior at training and prediction time, we
# need to create 2 distinct computation graphs that share the same weights.

# Create a graph for training
logits_train = conv_net(X, n_classes, dropout, reuse=False, is_training=True)
# Create another graph for testing that reuse the same weights, but has
# different behavior for 'dropout' (not applied).
logits_test = conv_net(X, n_classes, dropout, reuse=True, is_training=False)

# Define loss and optimizer (with train logits, for dropout to take effect)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Run the initializer
sess.run(init)

# Training cycle
for step in range(1, num_steps + 1):

    # Run optimization
    sess.run(train_op)

    if step % display_step == 0 or step == 1:
        # Calculate batch loss and accuracy
        # (note that this consume a new batch of data)
        loss, acc = sess.run([loss_op, accuracy])
        print("Step " + str(step) + ", Minibatch Loss= " + \
              "{:.4f}".format(loss) + ", Training Accuracy= " + \
              "{:.3f}".format(acc))

print("Optimization Finished!")